In [5]:
!wget https://github.com/DeepStudio-TW/torch-dataloader-tutorial/raw/main/data.csv

--2022-11-30 08:18:17--  https://github.com/DeepStudio-TW/torch-dataloader-tutorial/raw/main/data.csv
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DeepStudio-TW/torch-dataloader-tutorial/main/data.csv [following]
--2022-11-30 08:18:17--  https://raw.githubusercontent.com/DeepStudio-TW/torch-dataloader-tutorial/main/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13203465 (13M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]  12.59M  27.2MB/s    in 0.5s    

2022-11-30 08:18:19 (27.2 MB/s) - ‘data.csv’ saved [13203465/13203465]



In [12]:
import torch
import time
import pandas as pd
import torch.utils.data as tud
import numpy as np

* 一般的torch.utils.data 的 dataset，主要用來將資料進行載入，裡面一定要設置len() 跟 getitem()這兩個方法
1. len() 讓方法知道整個dataset的長度
2. getitem() 讓方法知道如何回傳數值

In [13]:
class NumDataset(tud.Dataset):
    '''Init: 使用變數宣告- data來源、總量、preprocess方法...等等'''
    def __init__(self,fname,number_length):
        super().__init__()
        self.df=pd.read_csv(fname).head(number_length)
        self.number_length=number_length
        
        self.data=self.df.data.values
        self.label=self.df.label.values
    '''一定要宣告長度，自訂義'''
    def __len__(self):
        return self.number_length
    '''定義回傳一筆資料的方式，input會是某個index, 輸出data, 以及label'''
    def __getitem__(self, idx):
        data=self.data[idx]
        label=self.label[idx]
        return data,label

In [15]:
dataset=NumDataset("/code/data.csv",5)

for d,l in dataset:
    print("d = ",type(d))
    print("l = ",l)

d =  <class 'numpy.float64'>
l =  0
d =  <class 'numpy.float64'>
l =  1
d =  <class 'numpy.float64'>
l =  2
d =  <class 'numpy.float64'>
l =  3
d =  <class 'numpy.float64'>
l =  4


In [4]:
L = len(dataset)
seta,setb,setc= tud.dataset.random_split(dataset,[3,1,1])

* 透過tud.dataset.random_split()可以將輸入的dataset拆分成指定的數量，並且按照指定的數量進行拆分

In [5]:
print("seta ========")
for d,l in seta:

    print(type(d),"d = ",d)
    print(type(l),"l = ",l)

for d,l in setb:
    print("setb ========")
    print(type(d),"d = ",d)
    print(type(l),"l = ",l)

for d,l in setc:
    print("setc ========")
    print(type(d),"d = ",d)
    print(type(l),"l = ",l)

seta ========
<class 'numpy.float64'> d =  -0.9327277456926928
<class 'numpy.int64'> l =  3
<class 'numpy.float64'> d =  0.3314613320867217
<class 'numpy.int64'> l =  0
<class 'numpy.float64'> d =  -1.515299754708565
<class 'numpy.int64'> l =  2
setb ========
<class 'numpy.float64'> d =  -0.7552733088729586
<class 'numpy.int64'> l =  4
setc ========
<class 'numpy.float64'> d =  0.2032911268458796
<class 'numpy.int64'> l =  1


* 透過tud.dataset,ConcatDataset()可以將兩個dataset進行連接，也可以直接透過 ”+” 達到一樣的效果，

In [6]:
testdata = tud.dataset.ConcatDataset((setb,setc))
len(testdata)
for d,l in testdata:
    print(type(d),"d = ",d)
    print(type(l),"l = ",l)

<class 'numpy.float64'> d =  -0.7552733088729586
<class 'numpy.int64'> l =  4
<class 'numpy.float64'> d =  0.2032911268458796
<class 'numpy.int64'> l =  1


* 透過tud.dataset,ConcatDataset()可以將兩個dataset進行連接，也可以直接透過 ”+” 達到一樣的效果，

In [7]:
testdata = setb+testdata
len(testdata)
for d,l in testdata:
    print(type(d),"d = ",d)
    print(type(l),"l = ",l)

<class 'numpy.float64'> d =  -0.7552733088729586
<class 'numpy.int64'> l =  4
<class 'numpy.float64'> d =  -0.7552733088729586
<class 'numpy.int64'> l =  4
<class 'numpy.float64'> d =  0.2032911268458796
<class 'numpy.int64'> l =  1


In [8]:
class NumIterDataset(tud.IterableDataset):
    '''Init: 使用變數宣告- data來源、總量、preprocess方法...等等'''
    def __init__(self,fname,number_length):
        super(NumIterDataset).__init__()
        self.fname=fname
        self.number_length=number_length
    '''長度不宣告也沒差還是可以跑，有時候可以是無窮迴圈dataset'''
#     def __len__(self):
#         return self.number_length
    '''定義iteration的方式，反正此dataset被call到之後開始跑iteration'''
    def __iter__(self):
        file_itr = open(self.fname)
        _=next(file_itr)
        for idx,line in enumerate(file_itr):
            if idx<self.number_length:
                d,l=line.split(",")
                yield float(d),int(l)
            else:
                break


In [11]:
iter_dataset=NumIterDataset("/code/data.csv",6)
for d,l in iter_dataset:
    print("d = ",d)
    print("l = ",l)

d =  0.3314613320867217
l =  0
d =  0.20329112684587966
l =  1
d =  -1.515299754708565
l =  2
d =  -0.9327277456926928
l =  3
d =  -0.7552733088729586
l =  4
d =  -0.12054968098895107
l =  5
